In [1]:
import pandas as pd
import numpy as np
import os as os


In [2]:
basics_url = 'https://datasets.imdbws.com/title.basics.tsv.gz'
aka_url ='https://datasets.imdbws.com/title.akas.tsv.gz'
rating_url='https://datasets.imdbws.com/title.ratings.tsv.gz'

In [3]:
basics = pd.read_csv(basics_url,sep='\t', low_memory=False)
akas = pd.read_csv(aka_url,sep='\t', low_memory=False)
ratings = pd.read_csv(rating_url,sep='\t', low_memory=False)

In [4]:
basics=basics.replace({'\\N':np.nan}) #replace with usable null format
akas=akas.replace({'\\N':np.nan})
ratings=ratings.replace({'\\N':np.nan})


In [5]:
basics.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,Carmencita,0,1894,NaN,1,"Documentary,Short"
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,NaN,5,"Animation,Short"
2,tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,0,1892,NaN,4,"Animation,Comedy,Romance"
3,tt0000004,short,Un bon bock,Un bon bock,0,1892,NaN,12,"Animation,Short"
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893,NaN,1,"Comedy,Short"


In [6]:
basics=basics.dropna(subset=['runtimeMinutes']) #drop nulls in runtime

In [7]:
basics=basics.dropna(subset=['genres']) #drop nulls in genres

In [8]:
documentary = basics['genres'].str.contains('documentary',case=False) #drop documentaries
basics = basics[~documentary]

In [9]:
basics=basics.loc[basics['titleType'] == 'movie'] #limit output to only movies

In [10]:
basics['startYear']=basics['startYear'].replace({np.nan:0}) #disregard null values
basics['startYear']=basics['startYear'].astype(object).astype(int) #convert object to int
basics=basics.loc[basics['startYear'].between(2000,2022)] 

In [11]:
akas.head()

,titleId,ordering,title,region,language,types,attributes,isOriginalTitle
0,tt0000001,1,Карменсіта,UA,NaN,imdbDisplay,NaN,0
1,tt0000001,2,Carmencita,DE,NaN,NaN,literal title,0
2,tt0000001,3,Carmencita - spanyol tánc,HU,NaN,imdbDisplay,NaN,0
3,tt0000001,4,Καρμενσίτα,GR,NaN,imdbDisplay,NaN,0
4,tt0000001,5,Карменсита,RU,NaN,imdbDisplay,NaN,0


In [12]:
akas=akas.loc[akas['region'] == 'US'] #keep only US titles

In [13]:
akas.head()

,titleId,ordering,title,region,language,types,attributes,isOriginalTitle
5,tt0000001,6,Carmencita,US,NaN,imdbDisplay,NaN,0
14,tt0000002,7,The Clown and His Dogs,US,NaN,NaN,literal English title,0
33,tt0000005,10,Blacksmith Scene,US,NaN,imdbDisplay,NaN,0
36,tt0000005,1,Blacksmithing Scene,US,NaN,alternative,NaN,0
41,tt0000005,6,Blacksmith Scene #1,US,NaN,alternative,NaN,0


In [14]:
# Filter the basics table down to only include the US by using the filter akas dataframe
keepers =basics['tconst'].isin(akas['titleId'])
keepers



34790       True
61089       True
67635       True
77929      False
86766       True
           ...  
9160989     True
9160998     True
9161037    False
9161082     True
9161166    False
Name: tconst, Length: 142252, dtype: bool

In [15]:
basics = basics[keepers]
basics



,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
34790,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001,NaN,118,"Comedy,Fantasy,Romance"
61089,tt0062336,movie,The Tango of the Widower and Its Distorting Mi...,El Tango del Viudo y Su Espejo Deformante,0,2020,NaN,70,Drama
67635,tt0069049,movie,The Other Side of the Wind,The Other Side of the Wind,0,2018,NaN,122,Drama
86766,tt0088751,movie,The Naked Monster,The Naked Monster,0,2005,NaN,100,"Comedy,Horror,Sci-Fi"
92731,tt0094859,movie,Chief Zabu,Chief Zabu,0,2016,NaN,74,Comedy
...,...,...,...,...,...,...,...,...,...
9160453,tt9914942,movie,Life Without Sara Amat,La vida sense la Sara Amat,0,2019,NaN,74,Drama
9160849,tt9915872,movie,The Last White Witch,My Girlfriend is a Wizard,0,2019,NaN,97,"Comedy,Drama,Fantasy"
9160989,tt9916170,movie,The Rehearsal,O Ensaio,0,2019,NaN,51,Drama
9160998,tt9916190,movie,Safeguard,Safeguard,0,2020,NaN,95,"Action,Adventure,Thriller"


In [16]:
keepers_rating =ratings['tconst'].isin(akas['titleId'])
ratings = ratings[keepers_rating]


In [17]:
# example making new folder with os
os.makedirs('Data/',exist_ok=True) 
# Confirm folder created
os.listdir("Data/")

['akas.csv.gz',
 'akas_resubmit.csv.gz',
 'ratings.csv.gz',
 'ratings_resubmit.csv.gz',
 'title_basics.csv.gz',
 'title_basics_resubmit.csv.gz']

In [18]:
basics.to_csv("Data/title_basics_resubmit.csv.gz",compression='gzip',index=False)


In [19]:
# Open saved file and preview again
basics = pd.read_csv("Data/title_basics_resubmit.csv.gz", low_memory = False)
basics.head()



,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001,NaN,118,"Comedy,Fantasy,Romance"
1,tt0062336,movie,The Tango of the Widower and Its Distorting Mi...,El Tango del Viudo y Su Espejo Deformante,0,2020,NaN,70,Drama
2,tt0069049,movie,The Other Side of the Wind,The Other Side of the Wind,0,2018,NaN,122,Drama
3,tt0088751,movie,The Naked Monster,The Naked Monster,0,2005,NaN,100,"Comedy,Horror,Sci-Fi"
4,tt0094859,movie,Chief Zabu,Chief Zabu,0,2016,NaN,74,Comedy


In [20]:
akas.to_csv("Data/akas_resubmit.csv.gz",compression='gzip',index=False)
# Open saved file and preview again
akas = pd.read_csv("Data/akas_resubmit.csv.gz", low_memory = False)
akas.head()



,titleId,ordering,title,region,language,types,attributes,isOriginalTitle
0,tt0000001,6,Carmencita,US,NaN,imdbDisplay,NaN,0.0
1,tt0000002,7,The Clown and His Dogs,US,NaN,NaN,literal English title,0.0
2,tt0000005,10,Blacksmith Scene,US,NaN,imdbDisplay,NaN,0.0
3,tt0000005,1,Blacksmithing Scene,US,NaN,alternative,NaN,0.0
4,tt0000005,6,Blacksmith Scene #1,US,NaN,alternative,NaN,0.0


In [21]:
ratings.to_csv("Data/ratings_resubmit.csv.gz",compression='gzip',index=False)
# Open saved file and preview again
ratings = pd.read_csv("Data/ratings_resubmit.csv.gz", low_memory = False)
ratings.head()



,tconst,averageRating,numVotes
0,tt0000001,5.7,1905
1,tt0000002,5.8,256
2,tt0000005,6.2,2518
3,tt0000006,5.1,173
4,tt0000007,5.4,783
